*********************************************************************************************************
# A Tour of Python 3  
version 1.0.1  
Authors: Phil Pfeiffer, Zack Bunch, and Feyisayo Oyeniyi  
East Tennessee State University  
Last updated June 2021  

Section 14.3: author John Pilson; ed. Phil Pfeiffer  
*********************************************************************************************************

# 14.  Command Line Processing  
 14.1 [Invoking a program from the command line, with arguments](#Command-Line-Processing-Invoking-With-Arguments)  
 14.2 [`__main__` and file interpretation](#Command-Line-Processing-Main-And-File-Interpretation)  
 14.3 [Argparse](#Command-Line-Processing-Argparse)  
 &ensp; 14.3.1 [Overview](#Command-Line-Processing-Argparse-Overview)    
 &ensp; 14.3.2 [ArgumentParser Objects](#Command-Line-Processing-Argparse-ArgumentParser)    
 &ensp; 14.3.3 [Specifying and Parsing Command Line Options](#Command-Line-Processing-Argparse-add_argument-and-parse_args)  
 &ensp; 14.3.4 [Other argparse functions](#Command-Line-Processing-Argparse-Other-argparse-functions)  

##  14.1  Invoking a program from the command line, with arguments <a name='Command-Line-Processing-Invoking-With-Arguments'></a>

Python implements standard (i.e., POSIX) conventions for command line processing. The following example shows the use of Python's `sys.argv` built-in to read command-line arguments. It also shows the use of Python's `subprocess` library to invoke a program from Jupyter: here, a sample program that illustrates command-line processing.

This example uses the following Python library resources:
-  `sys.argv` - a program's first (0th) argument is the name by which it was invoked; the balance are its remaining command line arguments.
-  `subprocess.run` - executes a program in a subshell, returning the program's final exit status 
   and, if `capture_output` is `True`, an object that captures that program's standard and error output streams. 

Note: The `subprocess.run` command shown below assumes that Python 3 is installed on the host system as `python`. If Python 3 is installed under a different name, change the command's first parameter to the correct name before running it. 

In [ ]:
# 14.1  Create, then run, a program, passing it command-line arguments and recovering its output

# ## import directives ##
#  subprocess.run - executes a program in a subshell
#  os.path.exists - check if a path expression denotes an object of some sort
#  os.remove - deletes specified file
#
import subprocess, os

# ## program constants ##
CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure

# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )

# - program to create
#
program_file_name = 'test.py'
program_content = [
    '# import directives\n',
    '# - sys.argv - list of command line arguments\n',
    '# - sys.stderr - error output\n',
    '# \n',
    'import sys\n',
    '\n',
    '# program constants \n',
    'FAILURE_EXIT = 1    # POSIX error code for program failure\n',
    '\n',
    '# supporting functions\n',
    '# \n',
    'def make_numeric(string):\n'
    '  try:\n',
    '    return int(string)\n', 
    '  except ValueError:\n',
    '    return float(string)\n', 
    '\n',
    'def plus(x, y): return x + y\n',
    '\n',
    'if __name__ == "__main__":\n',
    '  try:\n',
    '    assert len(sys.argv) >= 3, f"?? {sys.argv[0]}: insufficient arguments ({len(sys.argv)-1}); 2 required"\n',
    '    a, b = make_numeric(sys.argv[1]), make_numeric(sys.argv[2])\n',
    '    print( a, "+", b, "is", plus(a, b))\n',
    '  except Exception as exception:\n',
    '    exception_message = "" if str(exception) is None else str(exception)\n',
    '    print( f"?? {sys.argv[0]}: exiting", exception_message, file=sys.stderr )\n'
    '    exit( FAILURE_EXIT )\n'
]
trial_1 = [ '3', '4.2' ]
trial_2 = [ '3', 'four point two']
trial_3 = [ '3' ]
list_of_trials = [ trial_1, trial_2, trial_3 ]

# main proper
#
if os.path.exists( program_file_name ):
  print( f'{program_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( program_file_name, CREATE_NEW_FILE ) as program_fd:
   for line in program_content:
     program_fd.write( line )
  for (trial_number, this_trial) in enumerate(list_of_trials):
    print( 'executing sample program with argument list of ', this_trial )
    program_status = subprocess.run( [ 'python', program_file_name ] + this_trial, capture_output=True )
    print( 'return code is', program_status.returncode )
    standard_output = byte_seq_to_string( program_status.stdout )
    print( '> no standard output returned <' if standard_output == '' else 'standard output: ' + standard_output )
    error_output = byte_seq_to_string( program_status.stderr )
    print( '> no error output returned <' if error_output == '' else 'error output: ' + error_output )
    if trial_number+1 < len(list_of_trials): print( '---------------\n' )
  os.remove( program_file_name )

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 14.1.1:**

</span><span style='color:navy'>In the following code cell, redo this example so that it does the following: </span>
-  <span style='color:navy'>Identifies all command line arguments that can't be converted to numbers on the standard output, if any, with an appropriate error message.</span>
-  <span style='color:navy'>Writes the sum, minimum, maximum, and average of its numeric arguments to standard output,    writing an appropriate message to standard error if no numbers were supplied.</span>

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 14.1.2:**

</span><span style='color:navy'>In the following code cell, redo this example so that it does the following: </span>
-  <span style='color:navy'>Accepts one argument: a positive integer.</span>
-  <span style='color:navy'>Writes the sum of the first k values in the Taylor series expansion for pi to the standard output: i.e., 
   4 &ast; (1/1 - 1/3 + 1/5 - 1/7 + 1/9 - 1/11 ...).</span></span>

##  14.2  `__main__` and file interpretation <a name='Command-Line-Processing-Main-And-File-Interpretation'></a>

If a code is guarded with a check of whether the special variable `__name__` equals "`__main__`" that code will only execute when run from a command prompt. This `if __name__ == "__main__"` check is a common feature of supporting modules that include their own test code; it allows the modules' supporting routines to be referenced while assuring that the tests only run when the modules are invoked from a command line.

In [ ]:
# 14.2  Illustrating the use of 'if __name___ == "__main__" to guard test code

# ## import directives ##
#  subprocess.run - executes a program in a subshell
#  os.path.exists - check if a path expression denotes an object of some sort
#  os.remove - deletes specified file
#  importlib.invalidate_caches - invalidate the python cache
#  importlib.reload - reload a module
#
import subprocess, os, importlib

# ## program constants ##
CREATE_NEW_FILE = 'x'   # python open mode
FAILURE_EXIT = 1        # POSIX error code for program failure

# ## supporting functions ##
byte_seq_to_string = lambda byteseq: ''.join( chr(byte) for byte in byteseq )

# - program to create
#
module_name = 'test'
module_file_name = module_name + '.py'
module_content = [
    '# import directives\n',
    '# - sys.argv - list of command line arguments\n',
    '# - sys.stderr - error output\n',
    '# \n',
    'import sys\n',
    '\n',
    '# program constants \n',
    'FAILURE_EXIT = 1    # POSIX error code for program failure\n',
    '\n',
    '# supporting functions\n',
    '# \n',
    'def make_numeric(string):\n'
    '  try:\n',
    '    return int(string)\n', 
    '  except ValueError:\n',
    '    return float(string)\n', 
    '\n',
    'def plus(x, y): return x + y\n',
    '\n',
    'if __name__ == "__main__":\n',
    '  print( "-- tests would go here --" )\n'
]
trial_1 = [ '3', 'four point two']
trial_2 = [ '3', '4.2' ]
list_of_trials = [ trial_1, trial_2 ]

# main proper
#
if os.path.exists( module_file_name ):
  print( f'{module_file_name} already exists; please remove or rename it and rerun the example' )
else:
  with open( module_file_name, CREATE_NEW_FILE ) as module_fd:
    for line in module_content:
      module_fd.write( line )
  importlib.invalidate_caches() 
  dir_before_import = dir()
  exec(  f'import {module_name}' )
  exec(  f'importlib.reload( {module_name} )' ) 
  for (trial_number, (str1, str2)) in enumerate(list_of_trials):
    try:
      print( f'starting trial {trial_number}' )
      val1 = eval( f'{module_name}.make_numeric( {str1} )' )
      val2 = eval( f'{module_name}.make_numeric( {str2} )' )
      sum_ = eval( f'{module_name}.plus( {val1}, {val2} )' )
      print( f'sum of {str1} and {str2} is {sum_}' )
    except Exception as exception:
      exception_message = '' if str(exception) is None else str(exception)
      print( f'?? trial {trial_number} failed: ', exception_message )
  os.remove( module_file_name )

##  14.3  Argparse<a name='Command-Line-Processing-Argparse'></a>

### 14.3.1  Overview<a name='Command-Line-Processing-Argparse-Overview'></a>

Python's `argparse` module provides general purpose routines for parsing command line options in programs. An *option* is a sequence of strings that defines a program's operation: e.g., the files that it uses, time delays input between its operations, or how it issues messages.

Options are distinguished from other command line strings by a specially designated character prepended to the option’s name — the first string in the sequence. While argparse allows users to specify these prefix characters, programs typically use `-`, or possibly `-` and `+`. An option’s name is commonly followed by one or more parameters: e.g., filenames for document processing, seconds to pause between updating a display, or a string for formatting output.

`argparse` provides built in support for error checking, including tests for missing, invalid, and incorrectly formatted options. `argparse` also provides a standard help message that documents a command line’s expected format.

### 14.3.2  ArgumentParser<a name='Command-Line-Processing-Argparse-ArgumentParser'></a>

`argparse`'s `ArgumentParser` objects manage command line parsing for argparse. Parameters that can be used to adjust these objects include the following:
- `description` - the message to show in response to `argparse`'s built-in `--help` option
- `add_help` - enables the `--help` option; True by default
- `exit_on_error` - if True (default), treat parse errors as fatal
- `prefix_chars` - a list of prefix characters for option names; defaults to `-`
- `argument_default`- default values for missing options; can be useful when multiple options have float or integer values.
- `parents` -  when using multiple `ArgumentParser()` objects, `parents` can be set to a list of other `ArgumentParser()` objects. This allows the `ArgumentParser()` objects that use this parameter to inherit arguments from other `ArgumentParser()` objects stored within the list.

This next example illustrates `argparse`'s help message and error checking functions. `parse_args()` is described in the next section

In [ ]:
# 14.3.2  Illustrating argparse operation
# each parser is placed into a try-except block to avoid exiting

import argparse

try:
  parser1 = argparse.ArgumentParser()
  parser1.parse_args(["--help"])
except:
  print() 

try:
  parser2 = argparse.ArgumentParser()
  parser2.parse_args(["--foo"])
except:
  print()

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 14.3.2.1:**

</span><span style='color:navy'>In the following code cell, make the following changes to the preceding example:</span>
- <span style='color:navy'>Define one parser that accepts the arguments `--foo` and `--help`</span>
- <span style='color:navy'>Change the parser so that it doesn't raise an error when the unknown argument `--foo` is parsed</span>
- <span style='color:navy'>Update the help option to provide a description of the program when parsed as an argument</span>

### 14.3.3  Specifying and Parsing Command Line Options <a name='Command-Line-Processing-Argparse-add_argument-and-parse_args'></a>

ArgumentParser objects support an `add_argument()` method, which specifies command line options, including their names, syntax, and interpretation.

`add_argument()` can accept up to 11 named parameters defined below:
- `Name/flags` - The option's name. Multiple names can be defined for an option.
- `Action` - The action to do when the option is detected.  Actions are primarily defined for storing items and have predefined parameters such as `store`, `append`, and `count`.
- `Nargs` - the number of parameters that follow the option's name.  For example, to associate 2 parameters with an option, set Nargs to 2. This option — say, add — could then be invoked as `--add 5 6` to request the sum of 5 and 6.  Specifying `*` as a parameter causes `parse_args()` to treat any strings after the option name as additional parameters.
- `Const` - a constant value to pass as a parameter to the option's action. For example, multiplying by 3 can be done with a constant of 3 after accepting a number from command line.
- `Default` - the default value for the option, which is used when the associated parameters isn't found.
- `Type` - the parameter's data type: e.g., int for integers or float for floats.  Defaults to string if no Type is defined. Can reference a user-defined type conversion function.
- `Choices` - a list of legal values that the parameter an assume: e.g., `range(1, 11)` for a value between 1 and 10.
- `Required` - specifies that the option must be present. As a rule, this option should be avoided since options are meant to be optional.
- `Help` - The message describing the option when help is called at command line.
- `Dest` - The variable for storing parsed values of the option's parameters. It defaults to the option name when this parameter is missing.
- `Metavar` - an alternative name used to hide the name of the dest variable. Primarily used for data protection.

Some parameters, when combined, will cause `argparse` to throw an exception. One such pair is `Nargs` and `Const`, which can only be used together when `Nargs='?'`. This pair is used in a special case where `Nargs` will replace the constant value if an additional argument is given at command line.

An `ArgumentParser` object’s `parse_args()` method parses a string, checking for options specified by calls to add_argument(), confirming their types, and using the specified variables to reference their values. `parse_args()` defaults to parsing command lines; however, as seen above, arguments can also be passed as a list.  

In [ ]:
# 14.3.3.1 Illustrating the parameters of add_argument in an add function
# The code below accepts a two numbers and adds them together

import argparse

parser = argparse.ArgumentParser()
parser.add_argument("--add", nargs=2, type=int, default=0, choices=range(1, 11),
                    help="accepts two numbers and adds them together", dest="sum")
parsed_args = parser.parse_args(["--add", "10", "5"])

print(parsed_args.sum[0]+parsed_args.sum[1])

<span style='color:blue'>&#128073;&ensp;&ensp;**Exercise 14.3.3.1:**

</span><span style='color:navy'>In the following code cell, make the following changes to the preceding example:</span>
- <span style='color:navy'>Revise `--add` so that it accepts any number of arguments after it and adds them all together</span>
- <span style='color:navy'>Add an additional option `--subtract` that accepts two numbers, subtracting the second from the first</span>

### 14.3.4  Other argparse functions <a name='Command-Line-Processing-Argparse-Other-argparse-functions'></a>

Other helpful `argparse` functions and utilities include the following:
- `filetype` - a type parameter that can be present in `add_argument` that is used to parse files from arguments. Instead of getting a file name as a string from command line `filetype` can be used instead to store the contents of the file within a variable.
- `add_subparsers()` - an additional function useful in multi-purpose programs. To compartmentalize the code, an argument parser can be used as a subparser for a subset of a program's functions.
- `Parse_known_args()` - an alternative to `parse_args()` that parses only the arguments that the parser recognizes  and returns the rest as another list. This returned list can then be passed to additional parsers, including subparsers.
- `Add_argument_group()`- accepts a set of arguments and groups them when help is called. Useful for clarifying help messages in programs with a large number of arguments.